In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## EHR SUMMERIZER NOTEBOOK

#### OBJECTIVE :

#### 1.) Summarize electronic health records (EHR) for doctors.
#### 2.) Extract entities like symptoms, diagnoses, and medications.
#### 3.) Analyze patient sentiment for patient notes or interactions.
#### 4.) Provide a scalable pipeline for these tasks.

In [2]:
!pip install nlp
!pip install SpacyNLP
!pip install pipeline
!pip install healthchain
!pip install spacy
!python -m spacy download en_core_web_sm  # or another desired model
!pip install transformers
!pip install langchain
!pip install langchain_community
!pip install --upgrade langchain
!pip install langchain-core 
!pip install transformers 
!pip install spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.6 MB/s eta 0:00:0000:01
ERROR: Could not find a version that satisfies the requirement SpacyNLP (from versions: none)
ERROR: No matching distribution found for SpacyNLP
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.3/528.3 kB 13.9 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 44.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 76.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━

In [3]:
from langchain_core.output_parsers import StrOutputParser

# Replace JSONOutputParser with StrOutputParser
output_parser = StrOutputParser()
parsed_output = output_parser.parse("This is a sample output.")
print(parsed_output)

This is a sample output.


In [4]:
import json

class CustomJSONOutputParser:
    def parse(self, text):
        try:
            return json.loads(text)
        except json.JSONDecodeError as e:
            raise ValueError(f"Invalid JSON format: {e}")

# Example Usage
output_parser = CustomJSONOutputParser()
parsed_output = output_parser.parse('{"key": "value"}')
print(parsed_output)

{'key': 'value'}


In [5]:
from healthchain.io.containers import Document
from healthchain.pipeline.base import Pipeline
from healthchain.pipeline.components.integrations import (
    SpacyNLP,
    HFTransformer,
    LangChainLLM,
)

from langchain_core.prompts import PromptTemplate
# from langchain_core.output_parsers import JSONOutputParser
# from langchain.llms import HuggingFaceLLM
import nlp
from langchain_core.output_parsers import StrOutputParser
import langchain
print(langchain.__version__)

0.3.14


In [6]:
# Set up our components
spacy_component = SpacyNLP.from_model_id("en_core_web_sm")
huggingface_component = HFTransformer.from_model_id(
    model="distilbert-base-uncased-finetuned-sst-2-english",
    task="sentiment-analysis",
)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
# Set up LangChain with a FakeListLLM
fake_llm = FakeListLLM(responses=["HealthChain: Powerful NLP pipeline builder."])
# Define the prompt template
prompt = PromptTemplate.from_template("Summarize the following text: {text}")
# Create the LCEL chain
chain = prompt | fake_llm | StrOutputParser()
langchain_component = LangChainLLM(chain=chain, task="summarization")


NameError: name 'FakeListLLM' is not defined

In [ ]:
# Build our pipeline
pipeline = Pipeline()
pipeline.add_node(spacy_component)
pipeline.add_node(huggingface_component)
pipeline.add_node(langchain_component)
pipeline.build()


In [ ]:
# Process a document
doc = Document("HealthChain makes it easy to build powerful NLP pipelines!")
processed_doc = pipeline(doc)

In [ ]:
# Let's see what we got!
spacy_doc = processed_doc.nlp.get_spacy_doc()
sentiment = processed_doc.models.get_output("huggingface", "sentiment-analysis")
summary = processed_doc.models.get_output("langchain", "summarization")

In [ ]:
print(f"Tokens: {[token.text for token in spacy_doc]}")
print(f"Sentiment: {sentiment}")
print(f"Summary: {summary}")

In [ ]:
from healthchain.io.containers import Document
from healthchain.pipeline.base import Pipeline
from healthchain.pipeline.components.integrations import (
    SpacyNLP,
    HFTransformer,
    LangChainLLM,
)

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.llms import FakeListLLM

# Set up our components
spacy_component = SpacyNLP.from_model_id("en_core_web_sm")
huggingface_component = HFTransformer.from_model_id(
    model="distilbert-base-uncased-finetuned-sst-2-english",
    task="sentiment-analysis",
)

# Set up LangChain with a FakeListLLM
fake_llm = FakeListLLM(responses=["thi is the test for the EHR pipeline builder"])
# Define the prompt template
prompt = PromptTemplate.from_template("Summarize the following text: {text}")
# Create the LCEL chain
chain = prompt | fake_llm | StrOutputParser()
langchain_component = LangChainLLM(chain=chain, task="summarization")

# Build our pipeline
pipeline = Pipeline()
pipeline.add_node(spacy_component)
pipeline.add_node(huggingface_component)
pipeline.add_node(langchain_component)
pipeline.build()

# Process a document
doc = Document("diabitic , cancer , and joint pain patient")
processed_doc = pipeline(doc)

# Let's see what we got!
spacy_doc = processed_doc.nlp.get_spacy_doc()
sentiment = processed_doc.models.get_output("huggingface", "sentiment-analysis")
summary = processed_doc.models.get_output("langchain", "summarization")

print(f"Tokens: {[token.text for token in spacy_doc]}")
print(f"Sentiment: {sentiment}")
print(f"Summary: {summary}")

In [8]:
# Pipeline Workflow

# Data Ingestion: Load EHRs in various formats (e.g., text, JSON, or HL7).

# Preprocessing:

# Tokenization and sentence segmentation.

# Anonymization of sensitive data.

# NLP Tasks:

# Entity recognition for extracting symptoms, diagnoses, and medications.

# Summarization of medical records.

# Sentiment analysis of patient notes.

# Scalability: Implement scalable components using cloud platforms or distributed systems.

# User Interaction: Provide outputs in user-friendly formats via a web interface.

# Report Generation: Generate concise summaries and visual dashboards.



In [9]:
# Pipeline Components
# Data Ingestion and Preprocessing

# Input: Raw medical records in text or JSON format.
# Output: Cleaned and tokenized text.
# Named Entity Recognition (NER)

# Extract medical entities such as symptoms, medications, and diagnoses.
# Text Summarization

# Summarize long medical records into concise and readable formats.
# Sentiment Analysis

# Analyze patient notes to determine sentiment.
# Pipeline Orchestration

# Combine all components into a unified pipeline.


In [26]:

# import re

# def preprocess_text(text):
#     """
#     Cleans and preprocesses raw medical text.
#     """
#     # Remove sensitive information (e.g., names, IDs)
#     text = re.sub(r"Patient Name: \w+", "Patient Name: [REDACTED]", text)
#     text = re.sub(r"[^\w\s.,]", "", text)  # Remove special characters
#     text = text.strip()
#     return text

# # Example Input
# raw_text = """
# Patient Name: John Doe
# Symptoms: Fever, headache, fatigue.
# Diagnosis: Viral infection.
# Prescribed Medications: Paracetamol, Ibuprofen.
# """

# cleaned_text = preprocess_text(raw_text)
# print(cleaned_text)

In [25]:
# !pip install scispacy
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_lg-0.4.0.tar.gz

# import spacy

# # Load SciSpacy model
# nlp = spacy.load("en_core_sci_sm")
# spacy_component = SpacyNLP.from_model_id("en_core_web_sm")
# def extract_entities(text):
#     """
#     Extracts medical entities such as symptoms, medications, and diagnoses.
#     """
#     doc = nlp(text)
#     entities = [(ent.text, ent.label_) for ent in doc.ents]
#     return entities

# # Example Usage
# entities = extract_entities(cleaned_text)
# print("Extracted Entities:", entities)

In [5]:
!pip install nlp
!pip install SpacyNLP
!pip install pipeline
!pip install healthchain
!pip install spacy
!python -m spacy download en_core_web_sm  # or another desired model
!pip install transformers
!pip install langchain
!pip install langchain_community
!pip install --upgrade langchain
!pip install langchain-core 
!pip install transformers 
!pip install spacy
!pip install fitz
!pip install frontend 
!pip install PyMuPDF

ERROR: Could not find a version that satisfies the requirement SpacyNLP (from versions: none)
ERROR: No matching distribution found for SpacyNLP
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 81.7 MB/s eta 0:00:00:00:01:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
# !pip install transformers spacy
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from transformers import AutoModelForSeq2SeqLM
from transformers import pipeline as hf_pipeline  # Rename to avoid conflict
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import torch
from transformers import LongformerTokenizer, LongformerForSequenceClassification
import numpy as np
import pandas as pd
import re
import json
# import fitz  # PyMuPDF (for extracting text from PDF)
import os
# from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelForSeq2SeqLM
# # from typing import Union
import fitz


In [8]:
# Load ALL necessary Models
# Load Sentiment Analysis Pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Load BioBERT Model

ner_model = AutoModelForTokenClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
ner_tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")

# Create an NER Pipeline
# ner_pipeline = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer)
ner_pipeline = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer, grouped_entities=True)


# Load T5 Model for Summarization
summarization_model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")  # Use 't5-large' for better performance
summarization_tokenizer = AutoTokenizer.from_pretrained("t5-base")


# Load models for Context-Aware Sentiment Analysis
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
sentiment_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# Load Longformer model for handling long text
longformer_tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
longformer_model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096")

# Pipeline for Sentence Level Sentiment Analysis (DistilBERT)
sentiment_pipeline = pipeline("sentiment-analysis", model=sentiment_model, tokenizer=tokenizer)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# Function to process document in different formats
def extract_text_from_file(file_path: str) -> str:
    """
    Extracts text from a file based on its type (txt, pdf, json).
    """
    file_extension = os.path.splitext(file_path)[1].lower()
    
    if file_extension == '.txt':
        with open(file_path, 'r') as f:
            return f.read()
    elif file_extension == '.pdf':
        return extract_text_from_pdf(file_path)
    elif file_extension == '.json':
        return extract_text_from_json(file_path)
    else:
        raise ValueError(f"Unsupported file format: {file_extension}")


In [20]:
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_file: str) -> str:
    """
    Extracts text from a PDF file using PyMuPDF (fitz).
    """
    doc = fitz.open(pdf_file)
    text = ""
    for page in doc:
        text += page.get_text("text")
    return text

In [21]:
# Function to extract text from a JSON document
def extract_text_from_json(json_file: str) -> str:
    """
    Extracts text content from a JSON file.
    """
    with open(json_file, 'r') as f:
        data = json.load(f)
    # Assuming 'text' is the key containing the document content in the JSON
    return data.get("text", "")

In [22]:

def preprocess_text(text, to_lowercase=True):
    """
    Cleans and preprocesses raw medical text.
    Enhancements:
    - Anonymizes sensitive data (names, dates, IDs, phone numbers, emails).
    - Removes unwanted special characters.
    - Normalizes whitespace.
    - Converts to lowercase (optional).
    """
    # Anonymize patient names
    text = re.sub(r"Patient Name: \w+", "Patient Name: [REDACTED]", text)
    
    # Anonymize dates (e.g., 01/01/2023 or January 1, 2023)
    text = re.sub(r"\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b", "[DATE REDACTED]", text)
    text = re.sub(r"\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{1,2}, \d{4}\b", "[DATE REDACTED]", text, flags=re.IGNORECASE)
    
    # Anonymize IDs (e.g., Medical Record ID: 12345)
    text = re.sub(r"Medical Record ID: \d+", "Medical Record ID: [REDACTED]", text)
    
    # Anonymize phone numbers
    text = re.sub(r"\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}", "[PHONE REDACTED]", text)
    
    # Anonymize email addresses
    text = re.sub(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b", "[EMAIL REDACTED]", text)
    
    # Remove unwanted special characters
    text = re.sub(r"[^\w\s.,]", "", text)
    
    # Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()
    
    # Convert to lowercase if enabled
    if to_lowercase:
        text = text.lower()
    
    return text

# # Example Input
# raw_text = """
# Patient Name: Black Shadow
# Medical Record ID: 12345
# Phone: (123) 456-7890
# Email: black.shadow@example.com
# Diagnosis: The patient is diagnosed with cancer on 01/01/2023.

# """
# cleaned_text = preprocess_text(raw_text)
# print(cleaned_text)

In [23]:
def extract_entities(text):
    """
    Extracts entities using a pre-trained BioBERT model and a keyword-based matching for symptoms, diagnoses, and medications.
    
    Args:
        text (str): Input medical text for entity extraction.

    Returns:
        dict: Extracted entities grouped by type (e.g., symptoms, diagnoses, medications).
    """
    # Keyword-based lists for symptoms, diagnoses, and medications
    symptom_keywords = [
        "pain", "fever", "cough", "headache", "fatigue", "nausea", "dizziness", "sore", "chills", 
        "swelling", "tired", "vomiting", "diarrhea", "constipation", "drowsiness", "sweating", "appetite loss", 
        "confusion", "weakness", "bleeding", "shortness of breath", "chest pain", "back pain", "abdominal pain", 
        "muscle aches", "joint pain", "coughing up blood", "sore throat", "stomach cramps", "dysphagia", "anorexia", 
        "urinary retention", "joint stiffness", "itching", "rash", "dysuria", "hematuria", "palpitations", 
        "dizziness", "tremors", "memory loss", "insomnia", "night sweats", "cold hands", "fatigue", "burning sensation", 
        "bloating", "difficulty breathing", "weight loss", "weight gain", "numbness", "tingling", "lightheadedness", 
        "fainting", "ear pain", "tinnitus", "confusion", "uncontrolled shaking", "mood swings", "irregular heartbeat", 
        "chronic fatigue", "muscle weakness", "skin rash", "joint swelling", "shortness of breath", "palpitations", 
        "urinary incontinence", "frequent urination", "difficulty swallowing", "blurry vision", "yellowing skin", 
        "chronic cough", "leg swelling", "facial swelling", "neck stiffness", "stomach bloating", "difficulty sleeping", 
        "cold sweats", "feeling faint", "dehydration", "slow heartbeat", "congestion", "runny nose", "sinus headache", 
        "frequent headaches", "gastrointestinal discomfort", "liver pain", "hemoptysis", "chronic pain", "frequent thirst"
    ]
    
    diagnosis_keywords = [
        "cancer", "diabetes", "heart disease", "stroke", "asthma", "pneumonia", "arthritis", "hypertension", 
        "COPD", "depression", "anxiety", "influenza", "tuberculosis", "chronic obstructive pulmonary disease", "HIV", 
        "hypertensive heart disease", "gastritis", "stroke", "schizophrenia", "epilepsy", "osteoporosis", 
        "chronic kidney disease", "chronic pain", "fibromyalgia", "sleep apnea", "obesity", "bipolar disorder", 
        "Parkinson's disease", "dementia", "multiple sclerosis", "sickle cell anemia", "irritable bowel syndrome", 
        "hepatitis", "cirrhosis", "hypothyroidism", "hyperthyroidism", "urinary tract infection", "nephropathy", 
        "stroke", "Alzheimer's disease", "autoimmune disorder", "autoimmune disease", "HIV/AIDS", "cardiomyopathy", 
        "peripheral neuropathy", "gout", "lupus", "celiac disease", "pancreatitis", "colitis", "cancer", "renal failure", 
        "bipolar disorder", "schizophrenia", "Crohn's disease", "tuberculosis", "pneumonia", "mental illness", 
        "emphysema", "anemia", "Cushing's syndrome", "Addison's disease", "sickle cell disease", "sepsis", "septicemia", 
        "heart attack", "arrhythmia", "heart failure", "chronic fatigue syndrome", "malaria", "chickenpox", 
        "tuberculosis", "meningitis", "gonorrhea", "syphilis", "hepatitis C", "HIV infection", "gonorrhea", 
        "urinary tract infection", "psoriasis", "chronic bronchitis", "eczema", "dermatitis", "hypertension", "stroke", 
        "myocardial infarction", "cirrhosis", "brain tumor", "epileptic seizures", "lymphoma", "melanoma", "bacterial infection"
    ]
    
    medication_keywords = [
        "ibuprofen", "aspirin", "paracetamol", "amoxicillin", "metformin", "insulin", "acetaminophen", "naproxen", 
        "hydrocodone", "morphine", "antibiotics", "prednisone", "loratadine", "dextromethorphan", "fentanyl", 
        "sertraline", "fluoxetine", "alprazolam", "omeprazole", "lisinopril", "hydrochlorothiazide", "gabapentin", 
        "diclofenac", "cetirizine", "diphenhydramine", "clindamycin", "azithromycin", "levothyroxine", "amlodipine", 
        "warfarin", "hydroxychloroquine", "clopidogrel", "hydrocodone", "acetaminophen", "phenytoin", "topiramate", 
        "amlodipine", "losartan", "prednisolone", "carbamazepine", "valacyclovir", "antihistamine", "citalopram", 
        "zoloft", "paroxetine", "fluticasone", "albuterol", "salbutamol", "alendronate", "ranitidine", "esomeprazole", 
        "metoprolol", "lorazepam", "levothyroxine", "meloxicam", "simvastatin", "atorvastatin", "metformin", 
        "nifedipine", "lisinopril", "bupropion", "esomeprazole", "tramadol", "gabapentin", "cyclobenzaprine", 
        "clonazepam", "flunisolide", "captopril", "dexamethasone", "tamsulosin", "fluconazole", "amoxicillin", 
        "lorazepam", "digoxin", "hydrocodone", "doxycycline", "ketoconazole", "azithromycin", "miconazole", 
        "propranolol", "nystatin", "phenobarbital", "fluconazole", "dapoxetine", "nitroglycerin", "pantoprazole", 
        "cimetidine", "levofloxacin", "ivermectin", "sildenafil", "carvedilol", "calcium carbonate", "hydrochlorothiazide", 
        "spironolactone", "metoclopramide", "albuterol", "zolpidem", "tizanidine", "trazodone", "famotidine", 
        "prednisolone", "methylprednisolone", "hydrocortisone", "clomipramine", "dantrolene", "lamotrigine", 
        "valsartan", "topiramate", "methadone", "hydroxychloroquine"
    ]
    
    # Tokenize text into sentences (optional - helpful if text is too long)
    max_length = 512  # Typical max length for transformers
    sentences = [text[i:i + max_length] for i in range(0, len(text), max_length)]
    
    all_entities = []
    for sentence in sentences:
        # Run NER pipeline
        entities = ner_pipeline(sentence)
        all_entities.extend(entities)
    
    # Initialize categories
    categorized_entities = {"symptoms": [], "diagnoses": [], "medications": []}
    
    # Now classify based on the model's output and keyword matching
    for entity in all_entities:
        entity_type = entity.get("entity", "").lower()
        entity_word = entity.get("word", "").lower()

        # Map to categories using keywords
        if any(keyword in entity_word for keyword in symptom_keywords):
            categorized_entities["symptoms"].append(entity_word)
        elif any(keyword in entity_word for keyword in diagnosis_keywords):
            categorized_entities["diagnoses"].append(entity_word)
        elif any(keyword in entity_word for keyword in medication_keywords):
            categorized_entities["medications"].append(entity_word)

    return categorized_entities

# # Example Usage
# raw_text = """
# The patient is diagnosed with severe chest pain, high fever, and shortness of breath due to pneumonia. 
# Medications prescribed include ibuprofen for pain and amoxicillin for the infection.
# """

In [24]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# # Load pre-trained model and tokenizer for T5 summarization
# summarization_tokenizer = T5Tokenizer.from_pretrained("t5-small")
# summarization_model = T5ForConditionalGeneration.from_pretrained("t5-small")

def summarize_text(text, max_input_length=1024, max_summary_length=150, temperature=1.0, top_k=50, top_p=0.95, beam_width=8, no_repeat_ngram_size=2):
    """
    Summarizes text using a pre-trained T5 model with advanced parameters.
    
    Args:
        text (str): Input text to summarize.
        max_input_length (int): Maximum length of the input text (default 1024 tokens).
        max_summary_length (int): Maximum length of the summary (default 150 tokens).
        temperature (float): Temperature for sampling (default 1.0).
        top_k (int): Top-k sampling (default 50).
        top_p (float): Top-p sampling (default 0.95).
        beam_width (int): Number of beams for beam search (default 8).
        no_repeat_ngram_size (int): N-gram size to avoid repetition (default 2).
    
    Returns:
        str: Summarized text.
    """
    # Tokenize the input text with truncation if it's too long
    inputs = summarization_tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=max_input_length, truncation=True)
    
    # Generate the summary with advanced settings
    outputs = summarization_model.generate(
        inputs,
        max_length=max_summary_length, 
        min_length=25,
        length_penalty=2.0, 
        num_beams=beam_width,
        no_repeat_ngram_size=no_repeat_ngram_size, 
        early_stopping=True,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p
    )
    
    # Decode and return the summary
    summary = summarization_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

# # Example Usage
# print("===========================================================FINAL SUMMARY=============================================================")
# summary = summarize_text(cleaned_text)
# print("Summary:", summary)


In [25]:
# Longformer-based Context-Aware Sentiment Analysis (Handles long texts)
def analyze_sentiment_longformer(text, chunk_size=512, stride=256):
    """
    Analyzes sentiment of long texts using Longformer model to handle larger context.
    
    Args:
        text (str): Input text for sentiment analysis (could be large documents).
        chunk_size (int): Maximum length of tokenized input for Longformer.
        stride (int): Overlap between chunks to maintain context.
    
    Returns:
        dict: Sentiment analysis result with document-level context.
    """
    # Tokenize text in chunks
    inputs = longformer_tokenizer(text, return_tensors="pt", truncation=False, padding=True)
    input_ids = inputs['input_ids'][0]
    
    # If the length exceeds the maximum length, break the text into chunks
    if len(input_ids) > chunk_size:
        sentiment_scores = []
        
        # Sliding window approach: move with stride across the document
        for i in range(0, len(input_ids), chunk_size - stride):
            chunk = input_ids[i:i+chunk_size]
            # Ensure chunk is of the correct length
            if len(chunk) < chunk_size:
                chunk = chunk + [tokenizer.pad_token_id] * (chunk_size - len(chunk))

            # Perform sentiment analysis on the chunk
            inputs_chunk = {'input_ids': torch.tensor([chunk])}
            with torch.no_grad():
                outputs = longformer_model(**inputs_chunk)
                logits = outputs.logits
                sentiment_scores.append(torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()[0])

        # Average the scores from each chunk
        avg_scores = np.mean(sentiment_scores, axis=0)
        sentiment_label = "POSITIVE" if avg_scores[1] > avg_scores[0] else "NEGATIVE"
        sentiment_confidence = avg_scores[1] if avg_scores[1] > avg_scores[0] else avg_scores[0]
        return {'label': sentiment_label, 'confidence': sentiment_confidence}
    else:
        # If text is small, use the standard model
        sentiment = sentiment_pipeline(text)[0]
        return sentiment

# Function to analyze sentiment for mixed or large texts with context-awareness
def analyze_sentiment(text, long_text=False, chunk_size=512, stride=256):
    """
    Analyzes sentiment with context-awareness (Document level sentiment).
    
    Args:
        text (str): The text to analyze sentiment for.
        long_text (bool): Flag to use Longformer for large documents.
        chunk_size (int): Maximum length of each chunk.
        stride (int): Amount to overlap between chunks to maintain context.
    
    Returns:
        dict: Sentiment label and confidence.
    """
    if long_text:
        return analyze_sentiment_longformer(text, chunk_size, stride)
    else:
        # If it's small text, use regular sentiment analysis model
        sentiment = sentiment_pipeline(text)[0]
        return sentiment

# # Example Usage
# text = """
# Patient Name: John Doe
# Medical Record ID: 12345
# Diagnosis: The patient is diagnosed with breast cancer, undergoing chemotherapy.
# Symptoms: The patient reports feeling fatigued, nauseous, and experiencing abdominal pain.
# Prescribed Medications: Chemotherapy drugs and pain management.
# Treatment Plan: Start chemotherapy sessions next week and monitor for side effects.
# """

# # Analyzing sentiment with Document Level Context
# result = analyze_sentiment(text, long_text=True)
# print(f"Sentiment Analysis Result: {result}")

In [26]:
 # Main Pipeline Class
class MedicalNLPipeline:
    def process_document(self, file_path: str = None, prompt: str = None):
        """
        Processes a medical document in various formats (.txt, .json, .pdf) or custom prompt through the pipeline.
        """
        # Step 1: Check if a file or prompt is provided
        if file_path:
            extracted_text = extract_text_from_file(file_path)
        elif prompt:
            extracted_text = prompt
        else:
            raise ValueError("Either a file or a prompt must be provided.")
        
        # Step 2: Clean and preprocess the text
        cleaned_text = preprocess_text(extracted_text)

        # Step 3: Extract Entities using NER pipeline
        categorized_entities = extract_entities(cleaned_text)
        print("Categorized Entities:", categorized_entities)
        
        # Step 4: Summarize the text
        summary = summarize_text(cleaned_text)
        print("Summary:", summary)
        
        # Step 5: Perform sentiment analysis
        sentiment_result = analyze_sentiment(cleaned_text)
        print("Sentiment Analysis:", sentiment_result)

        # Step 6: Return all results as a dictionary for easy access
        return {
            'extracted_text': extracted_text,
            'cleaned_text': cleaned_text,
            'categorized_entities': categorized_entities,
            'summary': summary,
            'sentiment': sentiment_result
        }


In [27]:
# Loading The Final Pipeline MedicalNLPipeline:

In [43]:
# # Example Usage
# raw_text = ''

# # Instantiate the Medical NLP pipeline
# medical_nlp = MedicalNLPipeline()

# # Process the medical document
# result = medical_nlp.process_document(raw_text)

# # Output the results
# print("Final Output:")
# print(result)

# Example Usage
# file_path = "/kaggle/input/synthea-dataset-jsons-ehr/fhir/00/000/000148c5-990f-4e98-8ac0-0d869dd1f87a.json"  # Can be .txt, .json, or .pdf
prompt_text = "Patient presents with fever, headache, and sore throat. Diagnosis is likely viral infection. Treatment includes supportive care."

# Instantiate the Medical NLP pipeline
medical_nlp = MedicalNLPipeline()

# Process the medical document with file or prompt
result = medical_nlp.process_document(prompt_text)  # Or use: result = medical_nlp.process_document(prompt=prompt_text)

# Output the results
print("Final Output:")
print(result)


ValueError: Unsupported file format: .